In [1]:
import pandas as pd
import os.path
import scipy.io.wavfile
import numpy as np
# import cPickle as pkl
# import h5py

In [2]:
from dance.sampling import subsample_mean as subsample
# from dance.sampling import subsample_nth as subsample

In [3]:
# reload song list
meta_data = pd.read_csv("song_info_list.csv")
# meta_data = meta_data[:4]

labels_data = pd.read_csv("labels_list.csv")

In [4]:
# according to https://mediatum.ub.tum.de/doc/1138535/582915.pdf
# at least 15 seconds of a song are required to get accurate classification
def split_wave_to_samples(wave, sampling_rate=44100, sample_length=20, offset=5, downsample_size=100):
    # split the entire wave to windows of size samplig_rate*sample_length, offsets samples by offset*sampling_rate
    # defaults:
    # since different sampling rates are available, we assume a sampling rate of 44100 to get samples of the same size
    # each data sequence should contain roughly 10 seconds of sound
    # only low frequencies will contain information on the beat/rhythm of the song
    sample_size = sampling_rate * sample_length
    offset_size = sampling_rate * offset
    assert int(sampling_rate/downsample_size) == float(float(sampling_rate/downsample_size))
    samples = []
    for o in range(0, wave.shape[0] - sample_size + 1, offset_size):
        samples.append(subsample(wave[o:o+sample_size], downsample_size))
    return np.array(samples), downsample_size
    

In [5]:
def generate_samples(meta, labels, offset=5, downsample_size=100, mono_channels=True):
    # for each song, split into multiple samples and assign corresponding label
    data = []
    y = []
    for idx, song in meta.iterrows():
        # reload song list
        if os.path.isfile("songs/" + song["id"] + ".wav"):
            try:
                song_rate, song_wave = scipy.io.wavfile.read("songs/" + song["id"] + ".wav")
                # add file information to df
                meta.loc[idx, "rate"] = song_rate
                t, n = song_wave.shape
                meta.loc[idx, "time"] = t
                meta.loc[idx, "wav_dims"] = n
                song_labels = labels.loc[labels["id"]==song["id"]]
                samples, downsample_size = split_wave_to_samples(song_wave, offset=offset, downsample_size=downsample_size)
                for il, label in song_labels.iterrows():
                    for six, sample in enumerate(samples):
                        if mono_channels:
                            for channel in range(n): 
                                # each channel of the song should contain sufficient 
                                # information to retrieve the dance style
                                data.append({"label": label["label"], 
                                             "idx": six, 
                                             "channel":channel, 
                                             "id":song["id"], 
                                             "sample":sample[:, channel], 
                                             "rate": song_rate/downsample_size})
                        else:
                            data.append({"label": label["label"], 
                                         "idx": six, 
                                         "channel":"summed", 
                                         "id":song["id"], 
                                         "sample":np.sum(sample, axis=1),
                                         "rate": song_rate/downsample_size})
            except ValueError as err:
                pass
                # print(song["id"], "OS error: {0}".format(err))
    return pd.DataFrame(data), meta

In [6]:
# split data to training, and test
# since songs are randomly selected it's enough to use the first n songs for training
n_training = int(meta_data.shape[0]*0.5)
n_val = int(meta_data.shape[0]*0.2)
train_meta = meta_data[:n_training]
val_meta = meta_data[n_training:n_training+n_val]
test_meta = meta_data[n_training+n_val:]

In [7]:
# strongly downsampled sound data, separated channels
train_data, train_meta = generate_samples(train_meta, labels_data)
train_data.to_pickle("train_data.pkl")
val_data, val_meta = generate_samples(val_meta, labels_data)
val_data.to_pickle("val_data.pkl")
#train_data.to_csv('train_samples.csv', sep=';')
test_data, test_meta = generate_samples(test_meta, labels_data)
test_data.to_pickle("test_data.pkl")
#test_data.to_csv('test_samples.csv', sep=';')
#with open("train_data.pkl", "w") as f:
#    pkl.dump(train_data, f)
#    pkl.dump(test_data, f)

/usr/lib64/python2.7/site-packages/pandas/core/indexing.py:296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/usr/lib64/python2.7/site-packages/pandas/core/indexing.py:476: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [7]:
# strongly downsampled sound data, summed channels
train_data, train_meta = generate_samples(train_meta, labels_data, mono_channels=False)
train_data.to_pickle("train_data_summed.pkl")
val_data, val_meta = generate_samples(val_meta, labels_data, mono_channels=False)
val_data.to_pickle("val_data_summed.pkl")
test_data, test_meta = generate_samples(test_meta, labels_data, mono_channels=False)
test_data.to_pickle("test_data_summed.pkl")

/usr/lib64/python2.7/site-packages/pandas/core/indexing.py:296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/usr/lib64/python2.7/site-packages/pandas/core/indexing.py:476: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [ ]:
# high frequency sound data, separated channels
train_data, train_meta = generate_samples(train_meta, labels_data, downsample_size=4, offset=50)
train_data.to_pickle("train_data_high.pkl")
val_data, val_meta = generate_samples(val_meta, labels_data, downsample_size=4, offset=50)
val_data.to_pickle("val_data_high.pkl")
test_data, test_meta = generate_samples(test_meta, labels_data, downsample_size=4, offset=50)
test_data.to_pickle("test_data_high.pkl")

In [7]:
# high frequency sound data, summed channels
train_data, train_meta = generate_samples(train_meta, labels_data, downsample_size=4, offset=50, mono_channels=False)
train_data.to_pickle("train_data_summed_high.pkl")
val_data, val_meta = generate_samples(val_meta, labels_data, downsample_size=4, offset=50, mono_channels=False)
val_data.to_pickle("val_data_summed_high.pkl")
test_data, test_meta = generate_samples(test_meta, labels_data, downsample_size=4, offset=50, mono_channels=False)
test_data.to_pickle("test_data_summed_high.pkl")

/usr/lib64/python2.7/site-packages/pandas/core/indexing.py:296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/usr/lib64/python2.7/site-packages/pandas/core/indexing.py:476: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [51]:
# select data that was loaded only
meta_data = meta_data.dropna(axis=0, how="any")

,Unnamed: 0,youtube_link,id,rate,time,wav_dims
0,0,http://youtube.com/watch?v=KpsVD_GyJuc,KpsVD_GyJuc,44100.0,2824192.0,2.0
1,1,http://youtube.com/watch?v=ciPGSJW7CnY,ciPGSJW7CnY,48000.0,6244065.0,2.0
2,2,http://youtube.com/watch?v=GxOA5umDN18,GxOA5umDN18,44100.0,5293056.0,2.0
3,3,http://youtube.com/watch?v=PXSdVXstEAM,PXSdVXstEAM,44100.0,12353536.0,2.0
4,4,http://youtube.com/watch?v=FgDU17xqNXo,FgDU17xqNXo,48000.0,10487744.0,2.0
5,5,http://youtube.com/watch?v=6VgG4zl2QTg,6VgG4zl2QTg,44100.0,6250496.0,2.0
6,6,http://youtube.com/watch?v=LoyZpt4HFMY,LoyZpt4HFMY,48000.0,8290394.0,2.0
7,7,http://youtube.com/watch?v=4Gs_pVBeVYo,4Gs_pVBeVYo,44100.0,8224768.0,2.0
8,8,http://youtube.com/watch?v=vtCHTii1BMs,vtCHTii1BMs,44100.0,2679808.0,2.0
9,9,http://youtube.com/watch?v=SwJVBq2lwzI,SwJVBq2lwzI,48000.0,9186498.0,2.0


In [9]:
songs.shape

(933,)